<a href="https://colab.research.google.com/github/nhatduong01/SQL_on_BigQuery/blob/main/Advance/Analytic_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will work on `chicago_taxi_trips` dataset.

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# Set up the Project Environment
import os
os.environ["GCLOUD_PROJECT"] = "BigQuerySQLExercises"

In [3]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

data_ref = client.dataset(dataset_id = 'chicago_taxi_trips', project = 'bigquery-public-data')

dataset = client.get_dataset(data_ref)

tables = list(client.list_tables(dataset))

for table in tables:
  print(table.table_id)

taxi_trips


In [4]:
# get the table
table_ref = dataset.table('taxi_trips')

table = client.get_table(table_ref)

table.schema

[SchemaField('unique_key', 'STRING', 'REQUIRED', 'Unique identifier for the trip.', ()),
 SchemaField('taxi_id', 'STRING', 'REQUIRED', 'A unique identifier for the taxi.', ()),
 SchemaField('trip_start_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip started, rounded to the nearest 15 minutes.', ()),
 SchemaField('trip_end_timestamp', 'TIMESTAMP', 'NULLABLE', 'When the trip ended, rounded to the nearest 15 minutes.', ()),
 SchemaField('trip_seconds', 'INTEGER', 'NULLABLE', 'Time of the trip in seconds.', ()),
 SchemaField('trip_miles', 'FLOAT', 'NULLABLE', 'Distance of the trip in miles.', ()),
 SchemaField('pickup_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips.', ()),
 SchemaField('dropoff_census_tract', 'INTEGER', 'NULLABLE', 'The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips.', ()),
 SchemaField('pickup_community_area', 'INTEGER', 'NULLABLE', '

### 1) Seperate and order trips by community area:
We will return the Dataframe with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp` and
an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.

In [5]:
query1 = """
         SELECT pickup_community_area,
         trip_start_timestamp,
         trip_end_timestamp,
         RANK()
         OVER 
         (
           PARTITION BY pickup_community_area
           ORDER BY trip_start_timestamp
         ) AS trip_number
         FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
         WHERE DATE (trip_start_timestamp) = '2017-05-01'"""
query_job1 = client.query(query1, project = 'bigquerysqlexercises')

seperated_areas = query_job1.to_dataframe()

In [ ]:
seperated_areas.head(n = 50)

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,36.0,2017-05-01 04:00:00+00:00,2017-05-01 04:15:00+00:00,1
1,36.0,2017-05-01 08:45:00+00:00,2017-05-01 09:00:00+00:00,2
2,36.0,2017-05-01 09:45:00+00:00,2017-05-01 09:45:00+00:00,3
3,36.0,2017-05-01 10:00:00+00:00,2017-05-01 10:00:00+00:00,4
4,36.0,2017-05-01 10:45:00+00:00,2017-05-01 11:00:00+00:00,5
5,36.0,2017-05-01 12:00:00+00:00,2017-05-01 12:15:00+00:00,6
6,36.0,2017-05-01 12:00:00+00:00,2017-05-01 12:00:00+00:00,6
7,36.0,2017-05-01 12:30:00+00:00,2017-05-01 12:30:00+00:00,8
8,36.0,2017-05-01 12:30:00+00:00,2017-05-01 12:45:00+00:00,8
9,36.0,2017-05-01 13:00:00+00:00,2017-05-01 13:15:00+00:00,10


### 3) How much time elapses between trips?
We must shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.


In [11]:
query2 = """
         SELECT taxi_id,
         trip_start_timestamp,
         trip_end_timestamp,
         TIMESTAMP_DIFF
         (
           trip_start_timestamp,
           LAG(trip_end_timestamp)
           OVER
            (
             PARTITION BY taxi_id
             ORDER BY trip_start_timestamp
             ),
             MINUTE
           ) AS prev_break
         
         FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
         WHERE DATE(trip_start_timestamp) = '2017-05-01'"""
query_job2 = client.query(query2, project = "bigquerysqlexercises")
prev_break = query_job2.to_dataframe()
prev_break

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,0671aa9764799e2893118b9adeedb4f1c542bc9cbe32cd...,2017-05-01 06:15:00+00:00,2017-05-01 06:15:00+00:00,NaN
1,1196baed4ac6d505b1570924ec0cfd186711562a126278...,2017-05-01 00:15:00+00:00,2017-05-01 00:15:00+00:00,NaN
2,1196baed4ac6d505b1570924ec0cfd186711562a126278...,2017-05-01 00:45:00+00:00,2017-05-01 00:45:00+00:00,30.0
3,1196baed4ac6d505b1570924ec0cfd186711562a126278...,2017-05-01 01:00:00+00:00,2017-05-01 01:15:00+00:00,15.0
4,1196baed4ac6d505b1570924ec0cfd186711562a126278...,2017-05-01 02:15:00+00:00,2017-05-01 02:15:00+00:00,60.0
...,...,...,...,...
74718,b4d59da1d7b8c002094752251906d9a3bcc6c7ea71a15f...,2017-05-01 21:15:00+00:00,2017-05-01 21:15:00+00:00,30.0
74719,c56edff2010e56fcaddf073cbe76e5238a9343eaad1b7a...,2017-05-01 21:00:00+00:00,2017-05-01 21:15:00+00:00,1125.0
74720,ccdbd6a47f2ffa426f592d128b83df16f97944ffab447b...,2017-05-01 21:00:00+00:00,2017-05-01 21:15:00+00:00,15.0
74721,ce344554c17707363b83e98ced620bbb7787990dfe2cf9...,2017-05-01 21:15:00+00:00,2017-05-01 21:15:00+00:00,15.0
